# Preprocessing E-commerce Behavior Data for Customer Segmentation

Pada notebook ini, kita akan melakukan preprocessing dataset e-commerce (October 2019 – April 2020) dari Kaggle untuk menyiapkan fitur key seperti RFM (Recency, Frequency, Monetary) dan event counts sebelum menerapkan model clustering (k-means & k-medoids).


## 1. Setup Environment

Pastikan paket berikut sudah terinstall: pandas, numpy, scikit-learn, scikit-learn-extra (untuk k-medoids).


In [ ]:
%pip install pandas numpy

# 2. Imports

In [2]:
# 2. Imports
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler

## 3. Load Dataset

Sesuaikan path `data_path` dengan lokasi file CSV di sistem Anda.

In [3]:
## 3. Load Dataset
data_path = "./2019-Oct.csv"

# Load data dengan parsing tanggal
df = pd.read_csv(data_path, parse_dates=["event_time"])

# Tampilkan beberapa baris pertama untuk verifikasi
print(df.head())


MemoryError: Unable to allocate 1.27 GiB for an array with shape (4, 42448764) and data type object

## 4. Data Cleaning

- Buang baris tanpa `user_id` atau `user_session`.
- Hanya ambil event valid (view, cart, remove_from_cart, purchase).

In [3]:
# Drop missing user/session
df = df.dropna(subset=["user_id", "user_session"])

# Filter event_type jika perlu
df = df[df["event_type"].isin(["view", "cart", "remove_from_cart", "purchase"])]

print("Shape setelah cleaning:", df.shape)


## 5. Feature Engineering

1. **Event Counts** per user: view_count, cart_count, remove_count, purchase_count
2. **Recency**: hari sejak transaksi terakhir
3. **Frequency**: jumlah purchase events
4. **Monetary**: total nilai pembelian

In [6]:
# 5.1 Event Counts
event_counts = (
    df.pivot_table(
        index="user_id",
        columns="event_type",
        values="product_id",
        aggfunc="count",
        fill_value=0
    )
    .rename(columns={
        'view': 'view_count',
        'cart': 'cart_count',
        'remove_from_cart': 'remove_count',
        'purchase': 'purchase_count'
    })
)


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00+00:00,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00+00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01+00:00,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01+00:00,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04+00:00,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [5]:
# 5.2 Recency
group_purchase = df[df['event_type']=='purchase']
last_purchase = (
    group_purchase
    .groupby('user_id')['event_time']
    .max()
    .to_frame('last_purchase_date')
)
current_date = df['event_time'].max()
last_purchase['recency_days'] = (
    current_date - last_purchase['last_purchase_date']
).dt.days

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
42448759,2019-10-31 23:59:58+00:00,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193
42448760,2019-10-31 23:59:58+00:00,view,10800172,2053013554994348409,NaN,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8
42448761,2019-10-31 23:59:58+00:00,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd
42448762,2019-10-31 23:59:59+00:00,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204
42448763,2019-10-31 23:59:59+00:00,view,13300120,2053013557166998015,NaN,swisshome,155.73,525266378,6e57d2d7-6022-46e6-81d6-fa77f14cefd8


In [7]:
# 5.3 Frequency
temp = group_purchase.groupby('user_id')['event_time'].count()
frequency = temp.to_frame('frequency')

# 5.4 Monetary
monetary = (
    group_purchase
    .groupby('user_id')['price']
    .sum()
    .to_frame('monetary')
)

## 6. Gabungkan dan Tangani Nilai Hilang

In [8]:
# Join semua fitur
data = (
    event_counts
    .join(last_purchase['recency_days'], how='left')
    .join(frequency, how='left')
    .join(monetary, how='left')
)

In [9]:
# Isi NaN: recency hari maksimal jika belum pernah purchase, frequency/monetary 0
max_recency = last_purchase['recency_days'].max()

data.fillna({
    'recency_days': max_recency,
    'frequency': 0,
    'monetary': 0
}, inplace=True)

print("Data features sample:")
print(data.head())


## 7. Transformasi dan Scaling

- Log-transform fitur skewed untuk view, cart, remove, purchase, frequency, monetary
- Scale untuk k-means pada subset fitur RFM


In [ ]:
# Log-transform
to_log = ['view_count', 'cart_count', 'remove_count', 'purchase_count', 'frequency', 'monetary']
for col in to_log:
    data[f'log_{col}'] = np.log1p(data[col])

# Standard scaling untuk k-means pada RFM
features_kmeans = ['recency_days', 'log_purchase_count', 'log_frequency', 'log_monetary']
scaler = StandardScaler()
data_scaled = data.copy()
data_scaled[features_kmeans] = scaler.fit_transform(data_scaled[features_kmeans])

print("Contoh data scaled:")
print(data_scaled[features_kmeans].head())